# ($t = 0.1\pi$) Expected state after each step in DME using parameterized SWAP gate

In [25]:
import numpy as np
import scipy

t = np.pi*0.1
t_ZZ = t/2


Define partial trace and Hermitian conjugate function

In [26]:
def ptrace(matrix,index=1,n1=4,n2=4):
    # Partial trace function for density matrices.
    current_tensor=matrix.reshape([n1, n2, n1, n2])
    if index == 1:
        return np.trace(current_tensor, axis1=1, axis2=3)
    elif index == 2:
        return np.trace(current_tensor, axis1=0, axis2=2)
    else:
        print('Error')

def dagger(u):
    return np.asarray(np.matrix(u).H)

Pauli matrices etc

In [27]:
X = np.array([[0,1],
              [1,0]])

Y = np.array([[0,-1j],
              [1j,0]])

Z = np.array([[1,0],
              [0,-1]])

identity = np.array([[1,0],
              [0,1]])

swap = np.array([[1, 0, 0, 0],
            [0, 0, 1, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 1]])

_three_qubit_swap_q0_q2 = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

swap_02 = np.kron(_three_qubit_swap_q0_q2,identity)
swap_13 = np.kron(identity,_three_qubit_swap_q0_q2)

# Helper function to calculate the operator for the 4 qubit swap

def double_pauli(pauli):
    '''Take input Z, X or Y pauli matrices in numpy and output ZZ on qubit 02 + ZZ on qubit 13, etc'''
    pauli_02 = np.kron(np.kron(pauli,identity),np.kron(pauli,identity))
    pauli_13 = np.kron(np.kron(identity,pauli),np.kron(identity,pauli))
    return pauli_02 + pauli_13

# Define the operators
ZZ_4q = double_pauli(Z)
YY_4q = double_pauli(Y)
XX_4q = double_pauli(X)




Define the qubit initial density matrices

In [28]:
psi_plus = np.array([[0.5, 0.5],
                     [0.5, 0.5]])
psi_0 = np.array([[1, 0],
                  [0, 0]])
psi_bell = 1/2 * np.array([[1, 0, 0, 1],
                            [0, 0, 0, 0],
                            [0, 0, 0, 0],
                            [1, 0, 0, 1]])

rho_W = np.kron(psi_0,psi_0)
rho_M = psi_bell

psi = np.kron(rho_W,rho_M)

In [29]:
from two_qubit_decomposition import decompose_unitary, reconstruct_from_decomposition

decomposition = decompose_unitary(psi_bell)
decomposition


{'II': 0.25,
 'IX': 0.0,
 'IY': 0j,
 'IZ': 0.0,
 'XI': 0.0,
 'XX': 0.25,
 'XY': 0j,
 'XZ': 0.0,
 'YI': 0j,
 'YX': 0j,
 'YY': (-0.25+0j),
 'YZ': 0j,
 'ZI': 0.0,
 'ZX': 0.0,
 'ZY': 0j,
 'ZZ': 0.25}

In [30]:
reconstruct_from_decomposition(decomposition)

array([[0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j]])

Define the unitaries

In [31]:
# Parameterized SWAP

U_ZZ = scipy.linalg.expm(-1j*t_ZZ*(ZZ_4q))
U_YY = scipy.linalg.expm(-1j*t_ZZ*(YY_4q))
U_XX = scipy.linalg.expm(-1j*t_ZZ*(XX_4q))
U_XXZ = U_XX @ U_YY @ U_ZZ * np.exp(-1j * 2 *  t_ZZ)

# Native SWAP
U = scipy.linalg.expm(-1j*t*(swap_02+swap_13))

In [32]:
# Compare parameterized and native
np.linalg.norm(U_XXZ-U)

5.952883444891324e-16

## Iteration A

In [33]:
# Parameterized SWAP
rho_A = np.kron(rho_W,rho_M)
rho_A1 = U_ZZ @ rho_A @ dagger(U_ZZ)
rho_A2 = U_YY @ rho_A1 @ dagger(U_YY)
rho_A3 = U_XX @ rho_A2 @ dagger(U_XX)
# Native SWAP
rho_A3_swap = U @ rho_A @ dagger(U)

In [34]:
print('A initial work register state:')
print(np.round(ptrace(rho_A),3))
print('A1 (after Z0-Z2, Z1-Z3) work register state:')
print(np.round(ptrace(rho_A1),3))
print('A2 (after Y0-Y2, Y1-Y3) work register state:')
print(np.round(ptrace(rho_A2),3))
print('A3 (after X0-X2, X1-X3) work register state:')
print(np.round(ptrace(rho_A3),3))
print('---')
print('A3 (using native swap) work register state:')
print(np.round(ptrace(rho_A3_swap),3))

A initial work register state:
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
A1 (after Z0-Z2, Z1-Z3) work register state:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
A2 (after Y0-Y2, Y1-Y3) work register state:
[[ 0.952+0.j  0.   +0.j  0.   +0.j -0.019+0.j]
 [ 0.   +0.j  0.024+0.j -0.019+0.j  0.   +0.j]
 [ 0.   +0.j -0.019+0.j  0.024+0.j  0.   +0.j]
 [-0.019+0.j  0.   +0.j  0.   +0.j  0.001+0.j]]
A3 (after X0-X2, X1-X3) work register state:
[[ 0.909+0.j     0.   +0.j     0.   +0.j    -0.039+0.028j]
 [ 0.   +0.j     0.043+0.j     0.   -0.j     0.   +0.j   ]
 [ 0.   +0.j    -0.   -0.j     0.043+0.j     0.   +0.j   ]
 [-0.039-0.028j  0.   +0.j     0.   +0.j     0.005+0.j   ]]
---
A3 (using native swap) work register state:
[[ 0.909+0.j     0.   +0.j     0.   +0.j    -0.039+0.028j]
 [ 0.   +0.j     0.043+0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.043+0.j     0.   +0.j  

In [35]:
print('A work and 1st memory register initial state:')
print(np.round(rho_A,3))
print('A3 work and 1st memory register state using XXZ')
print(np.round(rho_A3,3))
print('A3 work and 1st memory register state using swap')
print(np.round(rho_A3_swap,3))

A work and 1st memory register initial state:
[[0.5 0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.

In [36]:
# Compare parameterized and native
np.linalg.norm(rho_A3_swap-rho_A3)

3.3082795310806904e-16

## Iteration B

In [37]:
# Parameterized SWAP
rho_B = np.kron(ptrace(rho_A3),rho_M)
rho_B1 = U_ZZ @ rho_B @ dagger(U_ZZ)
rho_B2 = U_YY @ rho_B1 @ dagger(U_YY)
rho_B3 = U_XX @ rho_B2 @ dagger(U_XX)
# Native SWAP
rho_B3_swap = U @ rho_B @ dagger(U)

In [38]:
print('B initial work register state:')
print(np.round(ptrace(rho_B),3))
print('B1 (after Z0-Z4, Z1-Z5) work register state:')
print(np.round(ptrace(rho_B1),3))
print('B2 (after Y0-Y4, Y1-Y5) work register state:')
print(np.round(ptrace(rho_B2),3))
print('B3 (after X0-X4, X1-X5) work register state:')
print(np.round(ptrace(rho_B3),3))
print('---')
print('B3 (using native swap) work register state:')
print(np.round(ptrace(rho_B3_swap),3))

B initial work register state:
[[ 0.909+0.j     0.   +0.j     0.   +0.j    -0.039+0.028j]
 [ 0.   +0.j     0.043+0.j     0.   -0.j     0.   +0.j   ]
 [ 0.   +0.j    -0.   -0.j     0.043+0.j     0.   +0.j   ]
 [-0.039-0.028j  0.   +0.j     0.   +0.j     0.005+0.j   ]]
B1 (after Z0-Z4, Z1-Z5) work register state:
[[ 0.909+0.j     0.   +0.j     0.   +0.j    -0.031+0.023j]
 [ 0.   +0.j     0.043+0.j     0.   -0.j     0.   +0.j   ]
 [ 0.   +0.j    -0.   -0.j     0.043+0.j     0.   +0.j   ]
 [-0.031-0.023j  0.   +0.j     0.   +0.j     0.005+0.j   ]]
B2 (after Y0-Y4, Y1-Y5) work register state:
[[ 0.869-0.j     0.   +0.j     0.   +0.j    -0.045+0.022j]
 [ 0.   +0.j     0.061+0.j    -0.014-0.j     0.   +0.j   ]
 [ 0.   +0.j    -0.014-0.j     0.061+0.j     0.   +0.j   ]
 [-0.045-0.022j  0.   +0.j     0.   +0.j     0.009+0.j   ]]
B3 (after X0-X4, X1-X5) work register state:
[[ 0.834-0.j     0.   +0.j     0.   +0.j    -0.059+0.046j]
 [ 0.   +0.j     0.075+0.j     0.   +0.j     0.   +0.j   ]
 [ 0.

In [39]:
print('B work and 2nd memory register initial state:')
print(np.round(rho_B,3))
print('B3 work and 2nd memory register state using XXZ')
print(np.round(rho_B3,3))
print('B3 work and 2nd memory register state using swap')
print(np.round(rho_B3_swap,3))

B work and 2nd memory register initial state:
[[ 0.455+0.j     0.   +0.j     0.   +0.j     0.455+0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j    -0.019+0.014j -0.   +0.j    -0.   +0.j
  -0.019+0.014j]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j    -0.   +0.j    -0.   +0.j    -0.   +0.j
  -0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j    -0.   +0.j    -0.   +0.j    -0.   +0.j
  -0.   +0.j   ]
 [ 0.455+0.j     0.   +0.j     0.   +0.j     0.455+0.j     0.   +0.j
   0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j     0.   +0.j    -0.019+0.014j -0.   +0.j    -0.   +0.j
  -0.019+0.014j]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j    

In [40]:
# Compare parameterized and native
np.linalg.norm(rho_B3_swap-rho_B3)

1.7547623883249698e-16

## Output

In [41]:
single_register_dm_01 = {
    'rho_W_dm_A1': ptrace(rho_A1),
    'rho_W_dm_A2': ptrace(rho_A2),
    'rho_W_dm_A3': ptrace(rho_A3),
    'rho_W_dm_B1': ptrace(rho_B1),
    'rho_W_dm_B2': ptrace(rho_B2),
    'rho_W_dm_B3': ptrace(rho_B3),
}



In [42]:
two_register_dm_01 = {
    'rho_W_M_dm_A3': rho_A3,
    'rho_W_M_dm_B3': rho_B3,
}

In [43]:
import csv

def write_to_csv(filename, data):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Key', 'Values'])
        for key, values in data.items():
            formatted_values = [[str(elem) for elem in row] for row in values]
            writer.writerow([key, formatted_values])

write_to_csv('t_01_DME_work.csv', single_register_dm_01)
write_to_csv('t_01_DME_work_n_mem.csv', two_register_dm_01)


In [44]:
import csv
import numpy as np

def load_from_csv(filename):
    data_loaded = {}
    with open(filename, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        for row in reader:
            key = row[0]
            values = eval(row[1])  # Convert string back to list
            data_loaded[key] = np.asarray(values)
    return data_loaded

single_register_dm_01_loaded = load_from_csv('t_01_DME_work.csv')
two_register_dm_01_loaded = load_from_csv('t_01_DME_work_n_mem.csv')

# Check the loaded dictionaries
print(single_register_dm_01_loaded['rho_W_dm_A1'])
print(two_register_dm_01_loaded['rho_W_M_dm_B3'])


[['(0.9999999999999999+0j)' '0j' '0j' '0j']
 ['0j' '0j' '0j' '0j']
 ['0j' '0j' '0j' '0j']
 ['0j' '0j' '0j' '0j']]
[['(0.4545339053710856-1.0842021724855044e-19j)' '0j' '0j'
  '(0.33331543133054525-0.24382410370009105j)' '0j'
  '(-4.336808689942018e-19+2.7755575615628914e-17j)'
  '(0.0718461294116258+0.10590377571205002j)' '0j' '0j'
  '(0.0718461294116258+0.10590377571205002j)'
  '(-4.336808689942018e-19+1.3877787807814457e-17j)' '0j'
  '(-0.04178734693114555+0.04604867654410305j)' '0j' '0j'
  '(-0.019313562148434212+0.014032124268112041j)']
 ['0j' '(0.0020619689257828926+0j)' '-3.1636593030691106e-20j' '0j'
  '(1.0899600256360993e-36-0.0063460878170489505j)' '0j' '0j'
  '(1.670978388828964e-36-0.0063460878170489505j)'
  '(-9.736742154287137e-20-4.602084967167094e-19j)' '0j' '0j'
  '(-9.736742154287137e-20-4.163926032952532e-19j)' '0j'
  '(0.002061968925782892+0j)' '-3.16365930306911e-20j' '0j']
 ['0j' '-3.1636593030691106e-20j' '(0.0020619689257828926+0j)' '0j'
  '(-9.736742154287137e-